In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(tf.float32, shape=(None, n_H0, n_W0, n_C0)) # (batch, height, width, channel)
    Y = tf.placeholder(tf.float32, shape=(None, n_y)) # (batch,classes)
    return X, Y
def initialize_parameters(n_C0, hpar):
    tf.set_random_seed(1)                              # so that your "random" numbers match ours
    parameters = {}
    n_C = n_C0
    for i in range(1, len(hpar['layer_conv_filter'])+1):
        n_H = hpar['layer_conv_filter'][i-1][0] # height
        n_W = hpar['layer_conv_filter'][i-1][1] # width
        n_F = hpar['layer_conv_filter'][i-1][2] # channel (the number of filter)
        parameters['W%d'%i] = tf.get_variable('W%d'%i, [n_H, n_W, n_C, n_F], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
        n_C = n_F # channel for the next element
    return parameters
def forward_propagation(X, n_y, parameters, hpar):
    L = len(hpar['layer_conv_filter'])
    units = {'P0': X}
    for i in range(1, L+1): #CONV2D -> RELU -> MAXPOOL LAYERS
        s_conv = [1] + hpar['layer_conv_stride'][i-1] + [1] # [1,s,s,1]
        p_conv = hpar['layer_conv_padding'][i-1]
        f_pool = [1] + hpar['layer_pool_filter'][i-1] + [1] # [1,f,f,1]
        s_pool = [1] + hpar['layer_pool_stride'][i-1] + [1] # [1,s,s,1]
        p_pool = hpar['layer_pool_padding'][i-1]
        units['Z%d'%i] = tf.nn.conv2d(units['P%d'%(i-1)], parameters['W%d'%i], strides = s_conv, padding = p_conv) # CONV2D: stride of 1, padding 'SAME'
        units['A%d'%i] = tf.nn.relu(units['Z%d'%i]) # RELU
        units['P%d'%i] = tf.nn.max_pool(units['A%d'%i], ksize = f_pool, strides = s_pool, padding = p_pool) # MAXPOOL: window 8x8, sride 8, padding 'SAME'
    Lf = len(hpar['hlayer_fully_connected'])
    units['F0'] = tf.contrib.layers.flatten(units['P%d'%L]) # FLATTEN
    for i in range(1, Lf + 1): # FULLY-CONNECTED LAYERS
        num_FC = hpar['hlayer_fully_connected'][i-1]
        units['F%d'%i] = tf.contrib.layers.fully_connected(units['F%d'%(i-1)], num_FC, activation_fn=tf.nn.relu, weights_initializer=tf.contrib.layers.xavier_initializer(seed=0))
    units['F%d'%(Lf+1)] = tf.contrib.layers.fully_connected(units['F%d'%(Lf)], n_y, activation_fn=None, weights_initializer=tf.contrib.layers.xavier_initializer(seed=0)) # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    return units['F%d'%(Lf+1)], units
def compute_cost(Z3, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=Y))
    return cost
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0): # modified date: 01/15/2018
    np.random.seed(seed)
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]
    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = max(1, m // mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    if (m >= mini_batch_size) and (m % mini_batch_size != 0): # Handling the end case (last mini-batch < mini_batch_size)
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    return mini_batches
def print_units(parameters, units, hpar):
    for i in range(1, len(hpar['layer_conv_filter'])+1):
        print('>> param shape = ', parameters['W%d'%i])
    print('>> layer shape = ', units['P0'])
    for i in range(1, len(hpar['layer_conv_filter'])+1): #CONV2D -> RELU -> MAXPOOL LAYERS
        print('>> layer shape = ', units['Z%d'%i])
        print('>> layer shape = ', units['A%d'%i])
        print('>> layer shape = ', units['P%d'%i])
    print('>> layer shape = ', units['F0'])
    for i in range(1, len(hpar['hlayer_fully_connected']) + 2): # FULLY-CONNECTED LAYERS
        print('>> layer shape = ', units['F%d'%i])

In [3]:
def convert_to_one_hot(labels, C):
    C = tf.constant(C, name='C') # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    one_hot_matrix = tf.one_hot(labels.squeeze(), C, axis=0)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    return one_hot
def model(X_train, Y_train, X_test, Y_test, hpar):
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []                                        # To keep track of the cost
    
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y) # Create Placeholders of the correct shape
    parameters = initialize_parameters(n_C0, hpar) # Initialize parameters
    Z3, units = forward_propagation(X, n_y, parameters, hpar) # Forward propagation: Build the forward propagation in the tensorflow graph
    print_units(parameters, units, hpar)
    cost = compute_cost(Z3, Y) # Cost function: Add cost function to tensorflow graph
    optimizer = tf.train.AdamOptimizer(learning_rate=hpar['learning_rate']).minimize(cost) # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    init = tf.global_variables_initializer() # Initialize all the variables globally
    with tf.Session() as sess: # Start the session to compute the tensorflow graph
        sess.run(init) # Run the initialization
        for epoch in range(hpar['num_epochs']): # Do the training loop
            minibatch_cost = 0.
            num_minibatches = int(m / hpar['minibatch_size']) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, hpar['minibatch_size'], seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch # Select a minibatch
                # IMPORTANT: The line that runs the graph on a minibatch.
                _ , temp_cost = sess.run([optimizer,cost], feed_dict={X:minibatch_X, Y:minibatch_Y}) # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                minibatch_cost += temp_cost / num_minibatches
            costs.append(minibatch_cost)
            # Print the cost every epoch
            if hpar['print_cost'] > 0 and epoch % hpar['print_cost'] == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(hpar['learning_rate']))
        plt.show()
        # Calculate the correct predictions
        predict_op = tf.argmax(Z3, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
        return train_accuracy, test_accuracy, parameters

In [4]:
def load_SINGS_data(dirpath = './datasets/coursera_dnn4_week1_Convolutional Model/datasets/'):
    import h5py
    train_dataset = h5py.File(dirpath+'train_signs.h5', "r")
    X_train_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    Y_train_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
    test_dataset = h5py.File(dirpath+'test_signs.h5', "r")
    X_test_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    Y_test_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    Y_train_orig = Y_train_orig.reshape((1, Y_train_orig.shape[0]))
    Y_test_orig = Y_test_orig.reshape((1, Y_test_orig.shape[0]))
    #-------------------------------------------------------
    X_train = X_train_orig/255. # Normalize image vectors
    X_test = X_test_orig/255.
    Y_train = convert_to_one_hot(Y_train_orig, len(classes)).T # Convert training and test labels to one hot matrices
    Y_test = convert_to_one_hot(Y_test_orig, len(classes)).T
    return X_train, Y_train, X_test, Y_test, classes
if True:
    X_train, Y_train, X_test, Y_test, classes = load_SINGS_data()
    print ("number of training examples = " + str(X_train.shape[0]))
    print ("number of test examples = " + str(X_test.shape[0]))
    print ("X_train shape: " + str(X_train.shape))
    print ("Y_train shape: " + str(Y_train.shape))
    print ("X_test shape: " + str(X_test.shape))
    print ("Y_test shape: " + str(Y_test.shape))    
    #-----------------------------------------------------------------
    hpar = {}
    hpar['layer_conv_filter'] = [[4,4,8], [2,2,16]] # [height, width, channel]
    hpar['layer_conv_stride'] = [[1,1], [1,1]] # [height, width]
    hpar['layer_conv_padding'] = ['SAME', 'SAME']
    hpar['layer_pool_filter'] = [[8,8], [4,4]] # [height, width]
    hpar['layer_pool_stride'] = [[8,8], [4,4]] # [height, width]
    hpar['layer_pool_padding'] = ['SAME', 'SAME']
    hpar['hlayer_fully_connected'] = []
    hpar['learning_rate'] = 0.009
    hpar['num_epochs'] = 100
    hpar['minibatch_size'] = 64
    hpar['print_cost'] = 5 # print epoch step; no print when it is zero
    _, _, parameters = model(X_train, Y_train, X_test, Y_test, hpar)

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)
>> param shape =  <tf.Variable 'W1:0' shape=(4, 4, 3, 8) dtype=float32_ref>
>> param shape =  <tf.Variable 'W2:0' shape=(2, 2, 8, 16) dtype=float32_ref>
>> layer shape =  Tensor("Placeholder:0", shape=(?, 64, 64, 3), dtype=float32)
>> layer shape =  Tensor("Conv2D:0", shape=(?, 64, 64, 8), dtype=float32)
>> layer shape =  Tensor("Relu:0", shape=(?, 64, 64, 8), dtype=float32)
>> layer shape =  Tensor("MaxPool:0", shape=(?, 8, 8, 8), dtype=float32)
>> layer shape =  Tensor("Conv2D_1:0", shape=(?, 8, 8, 16), dtype=float32)
>> layer shape =  Tensor("Relu_1:0", shape=(?, 8, 8, 16), dtype=float32)
>> layer shape =  Tensor("MaxPool_1:0", shape=(?, 2, 2, 16), dtype=float32)
>> layer shape =  Tensor("Flatten/flatten/Reshape:0", shape=(?, 64), dtype=float32)
>> layer shape =  Tensor("fully_connected/BiasAdd:0", shape=(?, 

KeyboardInterrupt: 

In [ ]:
"""
Cost after epoch 0: 1.921722
Cost after epoch 5: 1.612302
Cost after epoch 10: 0.840686
Cost after epoch 15: 0.661784
Cost after epoch 20: 0.581210
Cost after epoch 25: 0.471013
Cost after epoch 30: 0.410983
Cost after epoch 35: 0.450856
Cost after epoch 40: 0.366354
Cost after epoch 45: 0.337586
Cost after epoch 50: 0.377859
Cost after epoch 55: 0.288762
Cost after epoch 60: 0.270143
Cost after epoch 65: 0.280989
Cost after epoch 70: 0.255999
Cost after epoch 75: 0.251713
Cost after epoch 80: 0.229010
Cost after epoch 85: 0.252714
Cost after epoch 90: 0.199755
Cost after epoch 95: 0.210600
Tensor("Mean_1:0", shape=(), dtype=float32)
Train Accuracy: 0.90463
Test Accuracy: 0.8
"""